In [59]:
import torch 
import torch.nn as nn
import numpy
# Set up device
device =  torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)

Using device: cuda


In [ ]:
print(torch.cuda.memory_summary())
a = torch.rand(10, device=device)
print(torch.cuda.memory_summary())
del a
print(torch.cuda.memory_summary())
torch.cuda.empty_cache()
print(torch.cuda.memory_summary())


In [101]:
# Define a simple model
class SimpleModel(nn.Module):
    def __init__(self):
        super(SimpleModel, self).__init__()
        self.linear = nn.Linear(10, 1)

    def forward(self, x):
        return self.linear(x)

simple_model = SimpleModel().to(device)
x = torch.rand(10, device=device)
a = simple_model(x).to("cpu")
torch.cuda.empty_cache()
a

tensor([0.0758], grad_fn=<ToCopyBackward0>)

In [8]:
import numpy as np

# Example column vector
column_vector = np.array([[1], [2], [3]])  # 3x1 column vector

# 1. Use .reshape
row_vector_1 = column_vector.reshape(1, -1)  # Reshape to 1x3
print("Row Vector using reshape:")
print(row_vector_1)

# 2. Use .T (Transpose)
row_vector_2 = column_vector.T  # Transpose to 1x3
print("\nRow Vector using transpose:")
print(row_vector_2)

# 3. Flatten to 1D (optional, if you want a 1D array)
row_vector_3 = column_vector.flatten()
print("\nRow Vector as a 1D array:")
print(row_vector_3)


Row Vector using reshape:
[[1 2 3]]

Row Vector using transpose:
[[1 2 3]]

Row Vector as a 1D array:
[1 2 3]


In [5]:

import numpy as np

class InvertedPendulumEnv():
    """
    Custom Environment for Inverted Pendulum, compatible with OpenAI Gym.
    """
    metadata = {'render.modes': ['human']}

    def __init__(self):

        # State: [cart position, cart velocity, pole angle, pole angular velocity]
        self.state = None

        # Action space: [force applied to the cart (-1 or +1)]
        # self.action_space = spaces.Discrete(2)  # 0 = push left, 1 = push right

        # Observation space
        high = np.array([2.4,  # cart position
                         np.finfo(np.float32).max,  # cart velocity
                         12 * 2 * np.pi / 360,  # pole angle (radians)
                         np.finfo(np.float32).max])  # pole angular velocity
        # self.observation_space = spaces.Box(-high, high, dtype=np.float32)

        # Constants
        self.gravity = 9.8
        self.masscart = 1.0
        self.masspole = 0.1
        self.total_mass = self.masscart + self.masspole
        self.length = 0.5  # half the pole length
        self.polemass_length = self.masspole * self.length
        self.force_mag = 10.0
        self.tau = 0.02  # time interval for updates (seconds)

        self.theta_threshold_radians = 12 * 2 * np.pi / 360
        self.x_threshold = 2.4

    def reset(self):
        """
        Reset the environment to the initial state.
        """
        self.state = np.random.uniform(low=-0.05, high=0.05, size=(4,))
        return np.array(self.state, dtype=np.float32)

    def step(self, action):
        """
        Take one step in the environment.
        """
        x, x_dot, theta, theta_dot = self.state
        force = self.force_mag if action == 1 else -self.force_mag
        costheta = np.cos(theta)
        sintheta = np.sin(theta)

        # Equations of motion
        temp = (force + self.polemass_length * theta_dot**2 * sintheta) / self.total_mass
        theta_acc = (self.gravity * sintheta - costheta * temp) / \
                    (self.length * (4.0 / 3.0 - self.masspole * costheta**2 / self.total_mass))
        x_acc = temp - self.polemass_length * theta_acc * costheta / self.total_mass

        # Update state
        x = x + self.tau * x_dot
        x_dot = x_dot + self.tau * x_acc
        theta = theta + self.tau * theta_dot
        theta_dot = theta_dot + self.tau * theta_acc
        self.state = (x, x_dot, theta, theta_dot)

        # Check if episode is done
        done = x < -self.x_threshold or x > self.x_threshold or \
               theta < -self.theta_threshold_radians or theta > self.theta_threshold_radians

        reward = 1.0 if not done else 0.0  # Reward for staying upright

        return np.array(self.state, dtype=np.float32), reward, done, {}

    def render(self, mode='human'):
        """
        Render the environment.
        """
        print(f"Cart Position: {self.state[0]:.4f}, Pole Angle: {self.state[2]:.4f}")

    def close(self):
        """
        Close the environment.
        """
        pass


# Register and Test the Environment
if __name__ == "__main__":
    env = InvertedPendulumEnv()
    state = env.reset()
    print(f"Initial State: {state}")

    for _ in range(100):
        action = np.random.uniform(low=-1, high=1, size=(1))  # Random action
        state, reward, done, _ = env.step(action)
        env.render()
        if done:
            print("Episode finished.")
            break

    env.close()

Initial State: [ 0.01355184 -0.04007623 -0.00252802 -0.0147303 ]
Cart Position: 0.0128, Pole Angle: -0.0028
Cart Position: 0.0080, Pole Angle: 0.0027
Cart Position: -0.0006, Pole Angle: 0.0141
Cart Position: -0.0131, Pole Angle: 0.0313
Cart Position: -0.0295, Pole Angle: 0.0545
Cart Position: -0.0498, Pole Angle: 0.0838
Cart Position: -0.0740, Pole Angle: 0.1192
Cart Position: -0.1022, Pole Angle: 0.1610
Cart Position: -0.1343, Pole Angle: 0.2093
Cart Position: -0.1703, Pole Angle: 0.2644
Episode finished.


In [ ]:
import numpy as np
from utils import *

class Booster():

    render_mode = True

    def __init__(self) -> None:
        # State space
        self.state = None # x,y,z,phi,theta,psi,x_dot,y_dot,z_dot,phi_dot,theta_dot,psi_dot 

        # parameter 
        self.mass = 1 # mass 
        self.I0 = np.array([[1,0,0],
                            [0,1,0],
                            [0,0,1]]) # tensor
        self.I0_inv = np.array([[1,0,0],
                                [0,1,0],
                                [0,0,1]])

        self.height = 0.6 # height
        self.radius = 0.1 # radius
        self.mg = self.mass*np.array([0,0,-9.8]) # gravity 
        self.R = np.eye(3, dtype=np.float16) # rotation matrix

        # simulation 
        self.DT = 0.01
        self.vis = None
        self.scene_objects = None

        # Dynamic parameters
        self.p = np.zeros((3,1), dtype=np.float16)
        self.L = np.zeros((3,1), dtype=np.float16)
        self.w = np.zeros((3,1), dtype=np.float16)

        # limit for ending
        self.limit_threshold = np.degrees(30)
        self.x_threshold = -1.0
        self.y_threshold = -1.0
        self.y_threshold = 0.3

    def reset(self):
        """
        Reset the environment to the initial state.
        """
        # initial state
        [x,y,z] = [1.0, 1.0, 4.0]
        [phi,theta,psi,x_dot,y_dot,z_dot,phi_dot,theta_dot,psi_dot] = [0]*9
        self.state = np.array([x,y,z,
                               phi,theta,psi,
                               x_dot,y_dot,z_dot,
                               phi_dot,theta_dot,psi_dot], dtype=np.float16)
        self.R = R.from_euler('ZYX',np.array([phi,theta,psi])).as_matrix()

        # transformation of inital pose
        T = make_TF(x,y,z,self.R)

        # generate scene 
        self.scene_objects = create_scene(T)

        # create visualization 
        self.vis = o3d.visualization.Visualizer()
        self.vis.create_window(width=700, height=700)

        # add object to visualization 
        for obj in self.scene_objects:
            self.vis.add_geometry(obj)

        return self.state

    def step(self, action: np.array):
        """
        Update step for forward dynamics
        """
        # state 
        x,y,z,phi,theta,psi,x_dot,y_dot,z_dot,phi_dot,theta_dot,psi_dot  = self.state

        # F/T input
        validate_array_size(action)
        F_input = action[0:3]
        M_input = action[3:-1]

        # F/T in body frame
        F_b = F_input + self.mg
        r = np.array([0,0,-self.height/2]) # vector from C.O.M to force input
        M_b = M_input + np.cross(r, F_b)

        # Global input
        F_global = np.matmul(self.R, F_b)
        M_global = np.matmul(self.R, M_b)

        # Translation
        dp = F_global*self.DT
        self.p = self.p + dp
        x_dot = self.p/self.mass
        x = x + x_dot*self.DT

        # Rotation 
        dL = M_global*self.DT
        self.L = self.L + dL
        w = self.R*self.I0_inv*np.transpose(self.R)*self.L
        R_dot = np.dot(w_cross(w), self.R)
        self.R = self.R + R_dot*self.DT
        
        # Euler angle
        roll, pitch, yaw = o3d.geometry.get_euler_angles_from_rotation_matrix(self.R, order='XYZ')

        # Stack the state
        self.state = np.array([x,y,z,
                            roll,pitch,yaw,
                            x_dot,y_dot,z_dot,
                            w[0],w[1],w[2]], dtype=np.float16)
        
        # reward
        reward = 0

        # check collision 
        done = False

        return self.state, reward, done

    def render(self, time_delay=0.01):
        """
        Render the scene 
        """
        # transformation of body 
        T = make_TF(self.state[0],self.state[1],self.state[2],self.R)
        
        # update object position
        self.scene_objects[0].transform(T) # booster 
        self.scene_objects[1].transform(T) # body frame 
        
        # Update the visualization
        self.vis.update_geometry(self.scene_objects[0])
        self.vis.update_geometry(self.scene_objects[1])
        self.vis.poll_events()
        self.vis.update_renderer()

        # sleep to display
        time.sleep(time_delay)

        # convert back to inital 
        inv_transform = np.linalg.inv(T)
        self.scene_objects[0].transform(inv_transform) # booster 
        self.scene_objects[1].transform(inv_transform) # body frame 


    def close(self):
        self.vis.close()

        
         

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [ ]:
env = Booster()
state = env.reset()

for i in range(100):
    # generate action
    action = np.zeros(6, dtype=np.float16)
    
    # perform action
    state, reward, done = env.step(action)
    
    # render 
    env.render()

    # check done
    if done: break  

env.close()


d:\HomeworkSBU\Advance Dynamics\simulation-rocket-landing


ValueError: operands could not be broadcast together with shapes (2,) (3,) 

: 

In [ ]:
import open3d as o3d
import numpy as np
import time
from utils import *

# Load booster
booster =  o3d.io.read_triangle_mesh("./models/booster_model.STL")
booster.compute_vertex_normals()

# Transform to center of mesh 
transform_0 = np.eye(4)
transform_0[0,3] = -0.1
transform_0[1,3] = -0.1
transform_0[2,3] = -0.3
booster.transform(transform_0)

# Load chopstick 
chopstick = o3d.io.read_triangle_mesh("./models/chopstick_model.STL")
chopstick.compute_vertex_normals()

# Create the origin marker
origin = o3d.geometry.TriangleMesh.create_coordinate_frame(size=0.2)
body = o3d.geometry.TriangleMesh.create_coordinate_frame(size=0.2)
# body.translate([0, 0, 0.3])

arrow = draw_force(np.array([0]))

# Combine objects into a scene
scene_objects = [booster, chopstick, origin, body, arrow]

# Create a visualization window
vis = o3d.visualization.Visualizer()
vis.create_window(width=1000, height=1000)

for obj in scene_objects:
    vis.add_geometry(obj)

# # Set the viewing angle
# Get the view control
view_control = vis.get_view_control()

# Camera parameters
parameters = o3d.io.read_pinhole_camera_parameters("./ScreenCamera.json")
view_control.convert_from_pinhole_camera_parameters(parameters)

# Simulation parameters
velocity = 0.001  # m/s
distance = 1.0  # m
time_steps = int(distance / velocity)
step_time = 0.001 # seconds per step

# Simulate upward movement
for t in range(time_steps + 1):
    # Update the translation in the transformation matrix
    transform = np.eye(4)
    transform[2, 3] = velocity * t# Z-axis movement

    # Apply the transformation to the booster
    booster.transform(transform)
    body.transform(transform)

    # Update the visualization
    vis.update_geometry(booster)
    vis.update_geometry(body)
    vis.poll_events()
    vis.update_renderer()

    # Pause for the next frame
    time.sleep(step_time)

    # Convert back to origin
    inv_transform = np.linalg.inv(transform)
    booster.transform(np.linalg.inv(transform))
    body.transform(np.linalg.inv(transform))

# Close the visualization window after simulation
vis.run()
vis.destroy_window()

In [3]:
import open3d as o3d
import numpy as np

# Example: Load or create a mesh
mesh = o3d.geometry.TriangleMesh.create_sphere(radius=1.0)
mesh.compute_vertex_normals()

# Apply some transformations for demonstration
# Rotate and translate the mesh
rotation_matrix = o3d.geometry.get_rotation_matrix_from_xyz((np.pi / 4, np.pi / 4, 0))  # Rotate 45° on X and Y
mesh.rotate(rotation_matrix, center=(0, 0, 0))
mesh.translate([2, 3, 4])  # Translate to (2, 3, 4)

# Print original center and transformation
print("Original Center:", mesh.get_center())

# Reset the translation and rotation
# Step 1: Move the mesh back to the origin
translation_to_origin = -mesh.get_center()  # Compute translation to origin
mesh.translate(translation_to_origin, relative=False)

# Step 2: Reset the rotation (apply identity rotation matrix)
identity_rotation = np.eye(4)  # Identity matrix for no rotation
mesh.transform(identity_rotation)

# Verify the reset
print("New Center after Reset:", mesh.get_center())

# Visualize the reset mesh
o3d.visualization.draw_geometries([mesh], window_name="Reset Mesh")

Original Center: [2. 3. 4.]
New Center after Reset: [-2. -3. -4.]
[Open3D WARNING] GLFW Error: WGL: Failed to make context current: The handle is invalid. 


In [6]:
from utils import *

a = np.array([1,1,0])
b = np.array([2,3,4])

np.hstack((a,1))

array([1, 1, 0, 1])